In [1]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Device dignostics code

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Importing dataset

In [3]:
df = pd.read_csv(r"E:\Pytorch Datasets\Classification\wine+quality\winequality-white.csv", delimiter=';')

In [4]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [5]:
li = list(df.iloc[:, -1])
from collections import Counter
len(Counter(li))

7

Filtering Data

In [6]:
X = np.array(df.iloc[:, :-1])
y = np.array(df.iloc[:, -1])

In [7]:
type(X), type(y)

(numpy.ndarray, numpy.ndarray)

Converting numpy to tensor

In [8]:
X = torch.from_numpy(X).type(torch.float)
y = torch.from_numpy(y).type(torch.float)

In [9]:
type(X), type(y)

(torch.Tensor, torch.Tensor)

In [10]:
X.shape, y.shape

(torch.Size([4898, 11]), torch.Size([4898]))

Spliting training and testing data

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [12]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([3918, 11]),
 torch.Size([980, 11]),
 torch.Size([3918]),
 torch.Size([980]))

In [13]:
X_train, X_test, y_train, y_test = X_train.to(device), X_test.to(device), y_train.to(device), y_test.to(device)

Build a Linear model by inheriting nn.Module

In [14]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(in_features=11, out_features=32)
        self.layer2 = nn.Linear(in_features=32, out_features=64)
        self.layer3 = nn.Linear(in_features=64, out_features=64)
        self.layer4 = nn.Linear(in_features=64, out_features=8)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x
    
model = Model().to(device)

In [15]:
model.state_dict()

OrderedDict([('layer1.weight',
              tensor([[ 0.0608,  0.2182,  0.1040, -0.1690, -0.2866, -0.0166, -0.1911,  0.1346,
                        0.0939, -0.2838,  0.0043],
                      [-0.1032, -0.0808, -0.1374, -0.0154,  0.2914, -0.2671, -0.1776,  0.0959,
                        0.0740,  0.1071, -0.0094],
                      [ 0.0938, -0.0883,  0.1158,  0.1331,  0.1799,  0.2305, -0.1700,  0.1836,
                       -0.0399, -0.0441,  0.2818],
                      [-0.0570,  0.0131,  0.1590, -0.2251, -0.0226, -0.0937,  0.2001,  0.2398,
                        0.1867,  0.1259, -0.2143],
                      [ 0.0109,  0.2701, -0.1382, -0.0286,  0.1026, -0.0905, -0.0731,  0.2955,
                       -0.1831,  0.1475,  0.2403],
                      [-0.2572, -0.2141,  0.2719,  0.1408, -0.0313, -0.2405,  0.0668,  0.2313,
                        0.0404,  0.1504, -0.0407],
                      [ 0.1079,  0.2350,  0.0645,  0.1669, -0.0527,  0.0309,  0.1093, -0.2260

Testing everything is fine using dummy values

In [16]:
model.eval()

with torch.inference_mode():
    y_logits = model(X_test).squeeze()

In [17]:
y_logits.shape

torch.Size([980, 8])

In [18]:
activation = nn.Sigmoid()
y_pred = torch.round(activation(y_logits))

Choosing Loss and Optimizer

In [19]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    params=model.parameters(),
    lr=0.01
)

Writing Training and Test loops

In [20]:
y_train = y_train.long()

In [21]:
epochs = 101

for i in range(1, epochs):
    #training
    model.train()

    y_logits = model(X_train).squeeze()
    y_pred = torch.sigmoid(y_logits)

    train_loss = loss_fn(y_logits, y_train)
    train_acc = accuracy_score(y_train, y_pred)

    optimizer.zero_grad()

    train_loss.backward()

    optimizer.step()

    #testing 
    model.eval()
    with torch.inference_mode():
        test_logits = model(X_test).squeeze()
        test_pred = torch.round(torch.sigmoid(test_logits))

        test_loss = loss_fn( test_logits, y_test)
        test_acc = accuracy_score(y_test, test_pred)

    if i % 10 == 0:
        print(f"Epoch: {i} | Train Loss: {train_loss:.5f} | Train Accuracy: {train_acc:.2f} | Test Loss: {test_loss:.5f} | Test Accuracy: {test_acc:.2f}")

IndexError: Target 8 is out of bounds.